# Assignment 1

This notebook contains a set of exercises that will guide you through the different steps of this assignment. The aim of this assignment is to create and save a dataset containing information about every song in a given playlist by requesting data from Spotify's API. You will then use this dataset during the Artifical Intelligence I course to train a predictive model.

<div class="alert alert-danger"><b>Submission deadline:</b> Thursday, November 12, 20:00</div>

Read carefully the following instructions before starting the exercises.

### Instructions

- Write your code in the dedicated cells. You can use as many cells as you like. Just make sure to include all the necessary code before the corresponding test.

- This notebook is automatically graded. This means that there are several cells embedded into the notebook that take care of checking your code and grading it. It also means that it is important **to follow the instructions for each of the exercises** to make sure that yo do everything right.

- Checks use the ```nose``` library to assert different conditions.

- The tests for the introductory exercises will be open for you to see. This will help you understand how the pipeline works and check that you got the basics right. You can run these checks as many times as you want, **as long as you don't modify them**.  

- The tests for the remaining exercises will remain hidden. It is important that you **do not write any code, nor do you remove the cells left in blank** for this purpose. 

Before moving on, please run the following cell. You only need run it once in order to install the ```nose``` library.

In [727]:
pip install nose

## Getting client credentials

Spotify's API uses OAuth as an Authentication scheme. Hence, before starting to make requests, you need to get your client credentials to the Spotify API. 

![Dashboard](https://www.dropbox.com/s/cpfepk5fbq6ic5a/dashboard.png?raw=1)


To do so, you need to have a Spotify account (free or paid). If you don't have one yet, please create a free account before moving on. Once you do, head over to Spotify for Developers and open your [Dashboard](https://developer.spotify.com/dashboard/) and log in with your account. 

<img src="https://www.dropbox.com/s/afubgs4ar99uh80/app.png?raw=1" width="300">

Click on “CREATE AN APP”, choose a name and description for your project and work your way through the checkboxes. Don't worry about the actual name and description. The only thing we are interested in is getting the credentials.

![Credentials](https://www.dropbox.com/s/3mmxxeet61nha4l/credentials.png?raw=1)

Once your App has been created, you should see a “Client ID” and “Client Secret” on the left-hand side. These numbers correspond to your client credentials.

<div class="alert alert-info"><b>Exercise </b>Create two new variables, <i>client_id</i> and <i>client_secret</i>, that store your ID and Key, respectively</div>

In [728]:
# YOUR CODE HERE
client_id = '5f2b1e5aa99f45cb8cd5d0d4188507c2'
client_secret = 'b480e73533b440979bcc44f6d55816d0'

Great! We are good to go. Next step is getting an access token.

## Getting an access token

In order to access the various endpoints of the Spotify API, we need to pass an access token. 

To get one, we need to pass a ```POST``` request with our client credentials. This request will create a token resource in the server and respond back with it. We can build this ```POST``` request using ```requests``` library. remember that this library contains all the different methods available when interacting with an API. You can build the ```POST``` request by running the following cell:

In [729]:
import requests

# URL for token resource
auth_url = 'https://accounts.spotify.com/api/token'

# request body
params = {'grant_type': 'client_credentials',
          'client_id': client_id,
          'client_secret': client_secret}

# POST the request
auth_response = requests.post(auth_url, params)

Retrieve the body of the response in JSON format and store your token in a new variable called *access_token* by running the cell below. Take note of the different steps.

In [730]:
# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token to a new variable
access_token = auth_response_data['access_token']

Once you do, you can take a look at how your token looks like. You'll see that it is just a long alphanumeric string, like the client credentials you obtained in the previous step. This is your golden ticket to access Spotify's API. A copy of this string is now stored in the server, so that everytime to the API the server will check that the token you provide and the one it has in store match.

## Poking around

This API provides numerous endpoints to access things like album listings, artist information, playlists, even Spotify-generated audio analysis of individual tracks, which include their time signature or measurements such as their “danceability” or "loudness". You can take a look at all the information available by reading the [Docs](https://developer.spotify.com/documentation/web-api/reference/). In this assignment you will have to use different endpoints.

Independently of the specific data you want to retrieve, you need to send a properly formed GET request to the API server. As you should know by now, this request is composed of different elements. 

<img src="https://www.dropbox.com/s/hgb02k4h1mtdv22/header.png?raw=1" width="500">

As opposed to NASA's API, where we provided our API Key as part of the request body, Spotify's API expects you to include your access token in the requests header. There is a specific header called 'Authorization' for this purpose. Providing this information is sometimes tricky. Hence, the header has already been formatted for you. Just run the following cell to save save the header in a new variable so that you can use it later on.

In [731]:
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

In order to get a feel of how the API works, we will begin by making a ```GET``` requests to the ```audio-features``` endpoint to extract data for a specific track. In particular, let's retrieve all the information for Radiohead's **Creep**. 

The first thing you need is to identify the appropriate URL or path to direct your request to. The urls for all Spotify API endpoints follow the same structure. They all use the base URL for the API and are then defined as a concatenication of ```base_url + endpoint```. Sometimes, you will also need to provide some additional information as part of the path. In the case of ```audio-features```, it is enough with just the ```base_url``` and the ```endpoint``` name.

The ```base_url``` is defined below:

In [732]:
base_url = 'https://api.spotify.com/v1/'

And the endpoint for this case is, as we said, defined as:

In [733]:
endpoint = 'audio-features'

Hence, we can define the url as follows:

In [734]:
url = base_url + endpoint

Next thing we need is to fill in the request body. If you check the documentation you'll see that the ```audio-features``` endpoint takes the following query parameters.

<img src="https://www.dropbox.com/s/s4zs6wlue0u16cu/body.png?raw=1" width="500">

Hence, the final thing you need to extract data about Radiohead's Creep song is to locate its ```id```. This is its unique identifier. Spotify has unique ids for tracks, for artists, for albums, for playlists, etc.

![Creep](https://www.dropbox.com/s/kufj6ww2yn069gb/creep.png?raw=1)

You can get the ```id``` for any song by going to Spotify, looking for the song, clicking the “…” by the song name, then “Share” and then “Copy Spotify URI”. 

<div class="alert alert-warning">Note that this procedure also works for retrieving ids for artists, albums or any other resource type.</div>

This URI should be a string that includes something like **spotify:track:**, followed by an alphanumeric sequence. This sequence is the ID you are looking for.

<div class="alert alert-info"><b>Exercise </b>Create a new variable called <i>track_id</i> that stores the ID for Radiohead's song Creep.</div>

In [735]:
# YOUR CODE HERE
track_id = '6b2oQwSGFkzsMtQruIWm2p'

Now that we have the id, let's format the body of our request. As we did for NASA's API, we'll provide the body in dictionary form using a variable called *params*. Remember that the keys of this dictionary should correspond to the different query parameters defined in the documentation.

In [736]:
params = {'ids': [track_id]}

Now that everything is ready, you can run the actual GET request to retrieve the data.

<div class="alert alert-info"><b>Exercise </b>Write the code to make your get request using the requests library. When doing so, remember to pass the <i>url</i>, the <i>headers</i> and the <i>params</i> dictionary as arguments to the <i>get</i> functions. Store the response in a new variable called <i>creep</i>.</div>

In [737]:
# YOUR CODE HERE
creep = requests.get(url, headers=headers, params=params)

<div class="alert alert-warning">If you leave your notebook open for too long, your token might expire. When this happens, you will get an error {'error': {'status': 401, 'message': 'The access token expired'}} when making your request to the server. No worries. Just renew your token by executing the corresponding cell again and you should be good to go</div>

Finally, let's convert the response to JSON format to be able to manipulate it with greater ease.

<div class="alert alert-info"><b>Exercise </b>Write the code to convert the response to JSON format. Keep the name <i>creep</i>.</div>

In [738]:
# YOUR CODE HERE
creep = creep.json()

You can run the following cell to check if you obtained the right answer. If you get no error when running the cell it means that you did right. Otherwise, revise your code to ensure you get no error. You can run this cell as many times as you want, just **remember not to modify it**

In [739]:
from nose.tools import assert_equal
assert_equal(creep, {'audio_features': [{'danceability': 0.515,
                                         'energy': 0.43,
                                         'key': 7,
                                         'loudness': -9.935,
                                         'mode': 1,
                                         'speechiness': 0.0369,
                                         'acousticness': 0.0102,
                                         'instrumentalness': 0.000141,
                                         'liveness': 0.129,
                                         'valence': 0.104,
                                         'tempo': 91.841,
                                         'type': 'audio_features',
                                         'id': '6b2oQwSGFkzsMtQruIWm2p',
                                         'uri': 'spotify:track:6b2oQwSGFkzsMtQruIWm2p',
                                         'track_href': 'https://api.spotify.com/v1/tracks/6b2oQwSGFkzsMtQruIWm2p',
                                         'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6b2oQwSGFkzsMtQruIWm2p',
                                         'duration_ms': 238640,
                                         'time_signature': 4}]})

Congrats! You just made your first successful request to Spotify's API! 

Feel free to take a look at the information stored in the response. Pay special attention to the way in which the different information is stored. Remember that the JSON format works basically as a dictionary, so may want to check the slides on dictionaries before going further, to make sure you have them under control.

Once you are done, let's move on to some actual work!

## Getting data from a playlist

In the following exercise you will build a dataset containing data about different songs. You can either use a playlist of your own, or use the one we have created for this purpose. You can find our playlist in the following [link](https://open.spotify.com/playlist/4NVeFUEHBybfh3ITNG1b8n?si=js9BKt5aTOiCWMm_Cx4Vvg). If you prefer not to use this, feel free to complete the exercise with a playlist of your choosing. 

<div class="alert alert-info"><b>Exercise </b>Create a variable called <i>playlist_id</i> that stores the id of your playlist of choice.</div>

In [740]:
# YOUR CODE HERE
playlist_id = '45vumYFesN455BWS17dNKe'

Next step will be making a request to extract all the data bout your chosen playlist. Remember that you can take a look at all the information available at the different endpoints in Spotify's API by reading the [Docs](https://developer.spotify.com/documentation/web-api/reference/).

Locate the right endpoint for your query and read the Docs to find out how you should build your request. Don't use any of the optional arguments.

<div class="alert alert-info"><b>Exercise </b>Write the code to retrieve all the information about the chosen playlist in JSON form. Store the response in a new variable called <i>playlist</i>. When building your request, don't provide any information in the body. </div>

In [741]:
# YOUR CODE HERE
url_playlist = 'https://api.spotify.com/v1/playlists/45vumYFesN455BWS17dNKe/tracks'

In [742]:
playlist = requests.get(url_playlist, headers=headers)

In [743]:
playlist = playlist.json()

In [744]:
playlist

{'href': 'https://api.spotify.com/v1/playlists/45vumYFesN455BWS17dNKe/tracks?offset=0&limit=100',
 'items': [{'added_at': '2018-10-20T08:52:56Z',
   'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/alevecchi98'},
    'href': 'https://api.spotify.com/v1/users/alevecchi98',
    'id': 'alevecchi98',
    'type': 'user',
    'uri': 'spotify:user:alevecchi98'},
   'is_local': False,
   'primary_color': None,
   'track': {'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6O2zJ0tId7g07yzHtX0yap'},
       'href': 'https://api.spotify.com/v1/artists/6O2zJ0tId7g07yzHtX0yap',
       'id': '6O2zJ0tId7g07yzHtX0yap',
       'name': 'Mobb Deep',
       'type': 'artist',
       'uri': 'spotify:artist:6O2zJ0tId7g07yzHtX0yap'}],
     'available_markets': ['AD',
      'AE',
      'AL',
      'AR',
      'AT',
      'AU',
      'BA',
      'BE',
      'BG',
      'BH',
      'BO',
      'BR',
      'BY',
      'CA',
    

<div class="alert alert-warning">By default, Spotify's API only returns information about a maximum of 100 tracks in a playlist. There are ways to overcome this and I'll be happy to explain if you are interested. For now, though, let's keep it just like that. So, if your playlist of choice has more that 100 tracks, you'll get info about the first 100 of them.</div>

The following cells check whether your request is correct. Please **don't write any code here**. Just leave them blank

In [745]:
# LEAVE BLANK

In [746]:
# LEAVE BLANK

Once again, take your time to familiarize yourself with the data and how they are presented.

<div class="alert alert-info"><b>Exercise </b>Write the code to extract data about all the tracks included in your chosen playlist and save them into a pandas DataFrame object under the name <i>df</i>. The DataFrame should include the <i>album</i>, <i>artists</i>, <i>disc_number</i>, <i>duration_ms</i>, <i>explicit</i>, <i>name</i>, <i>popularity</i>, <i>release_date</i>, <i>track_number</i>, <i>uri</i>, <i>danceability</i>, <i>energy</i>, <i>key</i>, <i>loudness</i>, <i>mode</i>, <i>speechness</i>, <i>acousticness</i>, <i>instrulmentallness</i>, <i>liveness</i>, <i>valence</i> and <i>tempo</i> of every song. Use these same names as column names. In addition, your DataFrame should also include the total number of <i>followers</i>, the first listed <i>genre</i> and the <i>popularity</i> for the artists of each of the tracks. Store these data in columns called 'followers', 'genres' and 'artist_popularity'. The columns of your DataFrame should be ordered alphabetically. Use default index values.</div>

You can take a look at the following image for reference. 

![Dataframe](https://www.dropbox.com/s/42exa8hn43f9nyp/dataframe.png?raw=1)

In [747]:
# YOUR CODE HERE
import pandas as pd

In [748]:
album = []
artists = []
disc_number = []
duration_ms = []
explicit =[]
name = []
popularity = [] 
release_date = []
track_number = []
uri = []
danceability = []
energy = []
key = []
loudness = []
mode = []
speechiness = []
acousticness = []
instrumentalness = []
liveness = []
valence = []
tempo = []
followers = []
genres = []
artist_popularity = []

In [749]:
playlist.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [750]:
items = playlist['items']

In [751]:
playlist['items'][17]['track'].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track', 'track_number', 'type', 'uri'])

In [752]:
for item in items:
  artists.append(item['track']['artists'][0]['name'])

artists

['Mobb Deep',
 'Gang Starr',
 'Masta Ace',
 'Ice Cube',
 'Nas',
 'Dr. Dre',
 'Mobb Deep',
 'Mobb Deep',
 'Mobb Deep',
 'Canibus',
 'Canibus',
 'Dilated Peoples',
 'Jedi Mind Tricks',
 'Nas',
 'Nas',
 'Nas',
 'The Roots',
 'Vinnie Paz',
 'Vinnie Paz',
 'Jedi Mind Tricks',
 'Masta Ace',
 'Artifacts',
 'Masta Ace',
 'Common',
 'Masta Ace',
 'Mobb Deep',
 'Busta Rhymes',
 'Immortal Technique',
 'Immortal Technique',
 'Jamal',
 'Big Squig',
 'Masta Ace & Marco Polo',
 'Smif-N-Wessun',
 'Smif-N-Wessun',
 '704chop']

In [753]:
for item in items:
  album.append(item['track']['album']['name'])

album

['The Infamous',
 'Moment Of Truth',
 'A Long Hot Summer',
 'The Predator',
 'It Was Written',
 '2001',
 'Hell On Earth',
 'Murda Muzik',
 'The Infamous',
 'Rip The Jacker',
 'Rip The Jacker',
 'Expansion Team',
 'Violent By Design',
 'Illmatic',
 'The Lost Tapes',
 'The Lost Tapes',
 'The Tipping Point',
 'The Essential Collabo Collection Vol. 1',
 'The Essential Collabo Collection Vol. 2',
 'Violence Begets Violence',
 'Disposable Arts',
 "That's Them",
 'A Long Hot Summer',
 'Like Water For Chocolate',
 'A Long Hot Summer',
 'Murda Muzik',
 'Genesis',
 'Revolutionary Vol. 1',
 'Revolutionary Vol. 1',
 'Last Chance, No Breaks',
 'Keep It Going',
 'A Breukelen Story',
 'The All',
 'Smif N Wessun Reloaded',
 'Loyalty Means Everything']

In [754]:
for item in items:
  disc_number.append(item['track']['disc_number'])

disc_number

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [755]:
for item in items:
  duration_ms.append(item['track']['duration_ms'])

duration_ms

[325506,
 247333,
 207693,
 260000,
 234093,
 304026,
 274093,
 265493,
 224533,
 293400,
 438560,
 216000,
 229373,
 252600,
 235293,
 219466,
 315786,
 282000,
 265693,
 218293,
 253306,
 279906,
 225666,
 318800,
 251426,
 274906,
 274706,
 579293,
 197040,
 204199,
 279960,
 257049,
 222035,
 243506,
 112684]

In [756]:
for item in items:
  explicit.append(item['track']['explicit'])

explicit

[True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 True]

In [757]:
for item in items:
  popularity.append(item['track']['popularity'])

popularity

[74,
 56,
 52,
 76,
 65,
 60,
 66,
 62,
 69,
 31,
 26,
 51,
 49,
 57,
 35,
 50,
 0,
 14,
 10,
 55,
 48,
 24,
 29,
 44,
 26,
 40,
 0,
 58,
 42,
 26,
 20,
 16,
 23,
 19,
 48]

In [758]:
for item in items:
  track_number.append(item['track']['track_number'])

track_number

[15,
 8,
 5,
 7,
 2,
 22,
 11,
 10,
 3,
 5,
 11,
 2,
 8,
 9,
 2,
 8,
 4,
 5,
 10,
 6,
 13,
 15,
 6,
 9,
 17,
 14,
 12,
 11,
 3,
 9,
 1,
 16,
 3,
 11,
 1]

In [759]:
for item in items:
  uri.append(item['track']['uri'])

uri

['spotify:track:33ZXjLCpiINn8eQIDYEPTD',
 'spotify:track:1fotoYONO343JjbC8XvPSl',
 'spotify:track:2ouC5Zp9p98n6kAsXarudl',
 'spotify:track:2qOm7ukLyHUXWyR4ZWLwxA',
 'spotify:track:1AHfovSnGPVYKaahRtA0U6',
 'spotify:track:6WgxXdBZrtdQFGBcxQxpiv',
 'spotify:track:1RjU7UhRICmdFPKxCucLgq',
 'spotify:track:6nwAubYnLGl7F7cDuOoVLp',
 'spotify:track:7N1Vjtzr1lmmCW9iasQ8YO',
 'spotify:track:154o1htY1O9OJBt1gJfrfY',
 'spotify:track:55VdmFIyzDN6HBmUYshZKK',
 'spotify:track:4bHrZLxaOU1gfAz0rZAnCx',
 'spotify:track:0qw4HLV9qhjtImWR7oxZAO',
 'spotify:track:6DPrhGVJ1WTZvM9fKptnGe',
 'spotify:track:1gqniTFd41sW3Xk5IuigBB',
 'spotify:track:42qK1JWm8fDy3qHZbIZNuQ',
 'spotify:track:6lvkBfftJXIZMI6NDq3mjv',
 'spotify:track:3KTgkUnrtMD05s4y1zfGvd',
 'spotify:track:5TeOfQLvFKx3RcOSgv34N9',
 'spotify:track:6Z9syPcYlnUvpMgmjQydIG',
 'spotify:track:23RJg00mFONjIqYML6mtsE',
 'spotify:track:3KgN8zKTUOVSrwP83ice8t',
 'spotify:track:63B6RvjhocYifGQRNAwXWH',
 'spotify:track:6OE9S6XF0U1lNfeaUNSjYl',
 'spotify:track:

In [760]:
for item in items:
  name.append(item['track']['name'])

name

['Shook Ones, Pt. II',
 'Moment Of Truth',
 'Da Grind (Featuring Apocalypse)',
 'It Was A Good Day',
 'The Message',
 'The Message',
 'Hell on Earth (Front Lines)',
 'Quiet Storm',
 'Survival of the Fittest',
 'No Return',
 'Poet Laureate II',
 'Worst Comes To Worst',
 'I Against I',
 'Represent',
 'My Way',
 'Purple',
 'Guns Are Drawn',
 'Told U',
 'Liberal Arts',
 'Design in Malice',
 'Acknowledge',
 "Who's This ?",
 'H.O.O.D.',
 'The 6th Sense',
 'Wutuwankno',
 'Thug Muzik',
 'Holla',
 'Dance With the Devil',
 'Positive Balance',
 'Keep It Real',
 'Keep It Going',
 'Wanna Be (feat. Marlon Craft)',
 'DreamLand',
 'U Understand Me',
 'Loyalty Means Everything']

In [761]:
for item in items:
  release_date.append(item['track']['album']['release_date'])

release_date

['1995-04-25',
 '1998-03-31',
 '2004-08-03',
 '1992-11-17',
 '1996-07-02',
 '1999-11-16',
 '1996-11-19',
 '1999-08-17',
 '1995-04-25',
 '2003',
 '2003',
 '2001-10-23',
 '2004-05-25',
 '1994-04-19',
 '2002',
 '2002',
 '2004-07-13',
 '2016-12-02',
 '2016-12-02',
 '2011-10-25',
 '2005-02-15',
 '1997-01-01',
 '2004-08-03',
 '2000-01-01',
 '2004-08-03',
 '1999-08-17',
 '2001-11-13',
 '2001-09-18',
 '2001-09-18',
 '1995-01-01',
 '2019-03-01',
 '2018-11-09',
 '2019-02-22',
 '2005-10-18',
 '2019-08-12']



---



In [762]:
uri

['spotify:track:33ZXjLCpiINn8eQIDYEPTD',
 'spotify:track:1fotoYONO343JjbC8XvPSl',
 'spotify:track:2ouC5Zp9p98n6kAsXarudl',
 'spotify:track:2qOm7ukLyHUXWyR4ZWLwxA',
 'spotify:track:1AHfovSnGPVYKaahRtA0U6',
 'spotify:track:6WgxXdBZrtdQFGBcxQxpiv',
 'spotify:track:1RjU7UhRICmdFPKxCucLgq',
 'spotify:track:6nwAubYnLGl7F7cDuOoVLp',
 'spotify:track:7N1Vjtzr1lmmCW9iasQ8YO',
 'spotify:track:154o1htY1O9OJBt1gJfrfY',
 'spotify:track:55VdmFIyzDN6HBmUYshZKK',
 'spotify:track:4bHrZLxaOU1gfAz0rZAnCx',
 'spotify:track:0qw4HLV9qhjtImWR7oxZAO',
 'spotify:track:6DPrhGVJ1WTZvM9fKptnGe',
 'spotify:track:1gqniTFd41sW3Xk5IuigBB',
 'spotify:track:42qK1JWm8fDy3qHZbIZNuQ',
 'spotify:track:6lvkBfftJXIZMI6NDq3mjv',
 'spotify:track:3KTgkUnrtMD05s4y1zfGvd',
 'spotify:track:5TeOfQLvFKx3RcOSgv34N9',
 'spotify:track:6Z9syPcYlnUvpMgmjQydIG',
 'spotify:track:23RJg00mFONjIqYML6mtsE',
 'spotify:track:3KgN8zKTUOVSrwP83ice8t',
 'spotify:track:63B6RvjhocYifGQRNAwXWH',
 'spotify:track:6OE9S6XF0U1lNfeaUNSjYl',
 'spotify:track:

In [763]:
new_uri = ' '.join(uri).replace('spotify:track:','').split()
new_uri

['33ZXjLCpiINn8eQIDYEPTD',
 '1fotoYONO343JjbC8XvPSl',
 '2ouC5Zp9p98n6kAsXarudl',
 '2qOm7ukLyHUXWyR4ZWLwxA',
 '1AHfovSnGPVYKaahRtA0U6',
 '6WgxXdBZrtdQFGBcxQxpiv',
 '1RjU7UhRICmdFPKxCucLgq',
 '6nwAubYnLGl7F7cDuOoVLp',
 '7N1Vjtzr1lmmCW9iasQ8YO',
 '154o1htY1O9OJBt1gJfrfY',
 '55VdmFIyzDN6HBmUYshZKK',
 '4bHrZLxaOU1gfAz0rZAnCx',
 '0qw4HLV9qhjtImWR7oxZAO',
 '6DPrhGVJ1WTZvM9fKptnGe',
 '1gqniTFd41sW3Xk5IuigBB',
 '42qK1JWm8fDy3qHZbIZNuQ',
 '6lvkBfftJXIZMI6NDq3mjv',
 '3KTgkUnrtMD05s4y1zfGvd',
 '5TeOfQLvFKx3RcOSgv34N9',
 '6Z9syPcYlnUvpMgmjQydIG',
 '23RJg00mFONjIqYML6mtsE',
 '3KgN8zKTUOVSrwP83ice8t',
 '63B6RvjhocYifGQRNAwXWH',
 '6OE9S6XF0U1lNfeaUNSjYl',
 '0GIEMnTUU3i3Lixx8cbqx0',
 '1U5qqEm2St0V1TfCYvmdu4',
 '6lVM5MWbQuukUFfslPyHKT',
 '7MDUVH4ITohsIjdynRwCJp',
 '6C6kmwabmhtrPRQeKi0qdl',
 '64np9SGO0sZldyKTPwVcaH',
 '1vdrN37hpkK7mDifE4bNqj',
 '04462gRm1mWZQg2ivnGmor',
 '7vftM69ZhO5gBu0ElXuwBI',
 '3MbnEGwCIMaPyKlHO9J9n4',
 '3UnAM2kHBGx8YD4hUlycrp']

In [764]:
final_uri = ''
for x in new_uri:
  final_uri += x
  final_uri += ','

final_uri = final_uri[:-1]

final_uri

'33ZXjLCpiINn8eQIDYEPTD,1fotoYONO343JjbC8XvPSl,2ouC5Zp9p98n6kAsXarudl,2qOm7ukLyHUXWyR4ZWLwxA,1AHfovSnGPVYKaahRtA0U6,6WgxXdBZrtdQFGBcxQxpiv,1RjU7UhRICmdFPKxCucLgq,6nwAubYnLGl7F7cDuOoVLp,7N1Vjtzr1lmmCW9iasQ8YO,154o1htY1O9OJBt1gJfrfY,55VdmFIyzDN6HBmUYshZKK,4bHrZLxaOU1gfAz0rZAnCx,0qw4HLV9qhjtImWR7oxZAO,6DPrhGVJ1WTZvM9fKptnGe,1gqniTFd41sW3Xk5IuigBB,42qK1JWm8fDy3qHZbIZNuQ,6lvkBfftJXIZMI6NDq3mjv,3KTgkUnrtMD05s4y1zfGvd,5TeOfQLvFKx3RcOSgv34N9,6Z9syPcYlnUvpMgmjQydIG,23RJg00mFONjIqYML6mtsE,3KgN8zKTUOVSrwP83ice8t,63B6RvjhocYifGQRNAwXWH,6OE9S6XF0U1lNfeaUNSjYl,0GIEMnTUU3i3Lixx8cbqx0,1U5qqEm2St0V1TfCYvmdu4,6lVM5MWbQuukUFfslPyHKT,7MDUVH4ITohsIjdynRwCJp,6C6kmwabmhtrPRQeKi0qdl,64np9SGO0sZldyKTPwVcaH,1vdrN37hpkK7mDifE4bNqj,04462gRm1mWZQg2ivnGmor,7vftM69ZhO5gBu0ElXuwBI,3MbnEGwCIMaPyKlHO9J9n4,3UnAM2kHBGx8YD4hUlycrp'

In [765]:
params = {'ids': final_uri}

In [766]:
url_song = 'https://api.spotify.com/v1/audio-features/'
song = requests.get(url_song, params=params, headers=headers)
song = song.json()
song

{'audio_features': [{'acousticness': 0.0146,
   'analysis_url': 'https://api.spotify.com/v1/audio-analysis/33ZXjLCpiINn8eQIDYEPTD',
   'danceability': 0.763,
   'duration_ms': 325507,
   'energy': 0.786,
   'id': '33ZXjLCpiINn8eQIDYEPTD',
   'instrumentalness': 0.0114,
   'key': 10,
   'liveness': 0.0817,
   'loudness': -6.472,
   'mode': 0,
   'speechiness': 0.229,
   'tempo': 93.857,
   'time_signature': 4,
   'track_href': 'https://api.spotify.com/v1/tracks/33ZXjLCpiINn8eQIDYEPTD',
   'type': 'audio_features',
   'uri': 'spotify:track:33ZXjLCpiINn8eQIDYEPTD',
   'valence': 0.504},
  {'acousticness': 0.0157,
   'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1fotoYONO343JjbC8XvPSl',
   'danceability': 0.532,
   'duration_ms': 247333,
   'energy': 0.819,
   'id': '1fotoYONO343JjbC8XvPSl',
   'instrumentalness': 0,
   'key': 6,
   'liveness': 0.371,
   'loudness': -5.282,
   'mode': 0,
   'speechiness': 0.256,
   'tempo': 90.215,
   'time_signature': 4,
   'track_href': 'htt

In [767]:
features = song['audio_features']

In [768]:
for feature in features:
  danceability.append(feature['danceability'])

danceability

[0.763,
 0.532,
 0.729,
 0.798,
 0.614,
 0.711,
 0.875,
 0.79,
 0.813,
 0.676,
 0.744,
 0.801,
 0.717,
 0.702,
 0.603,
 0.865,
 0.932,
 0.411,
 0.758,
 0.358,
 0.729,
 0.782,
 0.696,
 0.688,
 0.617,
 0.717,
 0.782,
 0.673,
 0.477,
 0.598,
 0.588,
 0.626,
 0.556,
 0.745,
 0.791]

In [769]:
for feature in features:
  energy.append(feature['energy'])

energy

[0.786,
 0.819,
 0.619,
 0.744,
 0.543,
 0.608,
 0.658,
 0.728,
 0.703,
 0.869,
 0.943,
 0.732,
 0.743,
 0.832,
 0.825,
 0.595,
 0.644,
 0.853,
 0.736,
 0.94,
 0.705,
 0.658,
 0.699,
 0.827,
 0.686,
 0.668,
 0.789,
 0.871,
 0.837,
 0.675,
 0.669,
 0.71,
 0.472,
 0.705,
 0.398]

In [770]:
for feature in features:
  key.append(feature['key'])

key

[10,
 6,
 2,
 7,
 5,
 8,
 0,
 2,
 11,
 4,
 10,
 8,
 1,
 1,
 6,
 11,
 9,
 4,
 4,
 7,
 11,
 1,
 1,
 1,
 8,
 9,
 8,
 1,
 7,
 2,
 0,
 0,
 7,
 4,
 8]

In [771]:
for feature in features:
  loudness.append(feature['loudness'])

In [772]:
for feature in features:
  mode.append(feature['mode'])

In [773]:
for feature in features:
  speechiness.append(feature['speechiness'])

In [774]:
for feature in features:
  acousticness.append(feature['acousticness'])

In [775]:
for feature in features:
  instrumentalness.append(feature['instrumentalness'])

In [776]:
for feature in features:
  liveness.append(feature['liveness'])

In [777]:
for feature in features:
  valence.append(feature['valence'])

In [778]:
for feature in features:
  tempo.append(feature['tempo'])



---



In [779]:
url_artist = 'https://api.spotify.com/v1/artists'

In [780]:
id = []

In [781]:
playlist['items'][0]['track']['artists']

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/6O2zJ0tId7g07yzHtX0yap'},
  'href': 'https://api.spotify.com/v1/artists/6O2zJ0tId7g07yzHtX0yap',
  'id': '6O2zJ0tId7g07yzHtX0yap',
  'name': 'Mobb Deep',
  'type': 'artist',
  'uri': 'spotify:artist:6O2zJ0tId7g07yzHtX0yap'}]

In [782]:
for item in items:
  id.append(item['track']['artists'][0]['id'])

id

['6O2zJ0tId7g07yzHtX0yap',
 '5cMgGlA1xGyeAB2ctYlRdZ',
 '1wo9h8DP7M0M1orKuGZgWv',
 '3Mcii5XWf6E0lrY3Uky4cA',
 '20qISvAhX20dpIbOOzGK3q',
 '6DPYiyq5kWVQS4RGwxzPC7',
 '6O2zJ0tId7g07yzHtX0yap',
 '6O2zJ0tId7g07yzHtX0yap',
 '6O2zJ0tId7g07yzHtX0yap',
 '2lQvoGjN6jJfsAkaywtgrB',
 '2lQvoGjN6jJfsAkaywtgrB',
 '56OMwqr8qaLIRH4ZrvvdXq',
 '0PI3pXpUMScjweiw3IbADc',
 '20qISvAhX20dpIbOOzGK3q',
 '20qISvAhX20dpIbOOzGK3q',
 '20qISvAhX20dpIbOOzGK3q',
 '78xUyw6FkVZrRAtziFdtdu',
 '3rExe3A3FzAmYDlQuvW2Wx',
 '3rExe3A3FzAmYDlQuvW2Wx',
 '0PI3pXpUMScjweiw3IbADc',
 '1wo9h8DP7M0M1orKuGZgWv',
 '4hz8ohK9ESaFbJBAXvWJGL',
 '1wo9h8DP7M0M1orKuGZgWv',
 '2GHclqNVjqGuiE5mA7BEoc',
 '1wo9h8DP7M0M1orKuGZgWv',
 '6O2zJ0tId7g07yzHtX0yap',
 '1YfEcTuGvBQ8xSD1f53UnK',
 '7h8ja4JSORo2sXJPmCXRxa',
 '7h8ja4JSORo2sXJPmCXRxa',
 '1p9h1sy6bROcPGnLbBwLlO',
 '2Rse3rlXW6Ea8iRHxHiShS',
 '1qSvRXIiuE4WuRUhZtsFXA',
 '2p0Hf0MF8Nw8lwAXokczf0',
 '2p0Hf0MF8Nw8lwAXokczf0',
 '6awKBdY4phT3gd5WGGLraz']

In [783]:
final_id = ''
for x in id:
  final_id += x
  final_id += ','

final_id = final_id[:-1]

final_id

'6O2zJ0tId7g07yzHtX0yap,5cMgGlA1xGyeAB2ctYlRdZ,1wo9h8DP7M0M1orKuGZgWv,3Mcii5XWf6E0lrY3Uky4cA,20qISvAhX20dpIbOOzGK3q,6DPYiyq5kWVQS4RGwxzPC7,6O2zJ0tId7g07yzHtX0yap,6O2zJ0tId7g07yzHtX0yap,6O2zJ0tId7g07yzHtX0yap,2lQvoGjN6jJfsAkaywtgrB,2lQvoGjN6jJfsAkaywtgrB,56OMwqr8qaLIRH4ZrvvdXq,0PI3pXpUMScjweiw3IbADc,20qISvAhX20dpIbOOzGK3q,20qISvAhX20dpIbOOzGK3q,20qISvAhX20dpIbOOzGK3q,78xUyw6FkVZrRAtziFdtdu,3rExe3A3FzAmYDlQuvW2Wx,3rExe3A3FzAmYDlQuvW2Wx,0PI3pXpUMScjweiw3IbADc,1wo9h8DP7M0M1orKuGZgWv,4hz8ohK9ESaFbJBAXvWJGL,1wo9h8DP7M0M1orKuGZgWv,2GHclqNVjqGuiE5mA7BEoc,1wo9h8DP7M0M1orKuGZgWv,6O2zJ0tId7g07yzHtX0yap,1YfEcTuGvBQ8xSD1f53UnK,7h8ja4JSORo2sXJPmCXRxa,7h8ja4JSORo2sXJPmCXRxa,1p9h1sy6bROcPGnLbBwLlO,2Rse3rlXW6Ea8iRHxHiShS,1qSvRXIiuE4WuRUhZtsFXA,2p0Hf0MF8Nw8lwAXokczf0,2p0Hf0MF8Nw8lwAXokczf0,6awKBdY4phT3gd5WGGLraz'

In [784]:
params = {'ids': final_id}

In [785]:
info_artist = requests.get(url_artist, params=params, headers=headers)
info_artist = info_artist.json()
info_artist

{'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6O2zJ0tId7g07yzHtX0yap'},
   'followers': {'href': None, 'total': 1179038},
   'genres': ['east coast hip hop',
    'gangster rap',
    'hardcore hip hop',
    'hip hop',
    'queens hip hop',
    'rap'],
   'href': 'https://api.spotify.com/v1/artists/6O2zJ0tId7g07yzHtX0yap',
   'id': '6O2zJ0tId7g07yzHtX0yap',
   'images': [{'height': 640,
     'url': 'https://i.scdn.co/image/1d33cbed0798d07053b2773c9b46390d3fcfcfc9',
     'width': 640},
    {'height': 320,
     'url': 'https://i.scdn.co/image/5965f7bf761b984232a3b130079b0e24987fdd5e',
     'width': 320},
    {'height': 160,
     'url': 'https://i.scdn.co/image/771ef2f8896bb29e3289d92287de58c25c1e5353',
     'width': 160}],
   'name': 'Mobb Deep',
   'popularity': 71,
   'type': 'artist',
   'uri': 'spotify:artist:6O2zJ0tId7g07yzHtX0yap'},
  {'external_urls': {'spotify': 'https://open.spotify.com/artist/5cMgGlA1xGyeAB2ctYlRdZ'},
   'followers': {'href': None, '

In [786]:
artist_features = info_artist['artists']

In [787]:
info_artist['artists'][34]

{'external_urls': {'spotify': 'https://open.spotify.com/artist/6awKBdY4phT3gd5WGGLraz'},
 'followers': {'href': None, 'total': 6801},
 'genres': [],
 'href': 'https://api.spotify.com/v1/artists/6awKBdY4phT3gd5WGGLraz',
 'id': '6awKBdY4phT3gd5WGGLraz',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab67616d0000b273afb3e37c3455fbd845789b4e',
   'width': 640},
  {'height': 300,
   'url': 'https://i.scdn.co/image/ab67616d00001e02afb3e37c3455fbd845789b4e',
   'width': 300},
  {'height': 64,
   'url': 'https://i.scdn.co/image/ab67616d00004851afb3e37c3455fbd845789b4e',
   'width': 64}],
 'name': '704chop',
 'popularity': 44,
 'type': 'artist',
 'uri': 'spotify:artist:6awKBdY4phT3gd5WGGLraz'}

In [788]:
followers = []

for artist_feature in artist_features:
  followers.append(artist_feature['followers']['total'])

print(followers)

[1179038, 619470, 136252, 3963746, 2115114, 5233102, 1179038, 1179038, 1179038, 62089, 62089, 158193, 285016, 2115114, 2115114, 2115114, 964210, 198178, 198178, 285016, 136252, 49192, 136252, 662337, 136252, 1179038, 1624013, 362714, 362714, 3364, 201, 4207, 126398, 126398, 6801]


In [789]:
genres = []

for artist_feature in artist_features:
  if len(artist_feature['genres']) != 0:
     genres.append(artist_feature['genres'][0])
  else:
    genres.append('none')

print(genres)

['east coast hip hop', 'alternative hip hop', 'alternative hip hop', 'conscious hip hop', 'conscious hip hop', 'g funk', 'east coast hip hop', 'east coast hip hop', 'east coast hip hop', 'battle rap', 'battle rap', 'alternative hip hop', 'conscious hip hop', 'conscious hip hop', 'conscious hip hop', 'conscious hip hop', 'alternative hip hop', 'conscious hip hop', 'conscious hip hop', 'conscious hip hop', 'alternative hip hop', 'alternative hip hop', 'alternative hip hop', 'alternative hip hop', 'alternative hip hop', 'east coast hip hop', 'east coast hip hop', 'conscious hip hop', 'conscious hip hop', 'none', 'none', 'none', 'alternative hip hop', 'alternative hip hop', 'none']


In [790]:
artist_popularity = []

for artist_feature in artist_features:
  artist_popularity.append(artist_feature['popularity'])

print(artist_popularity)

[71, 66, 54, 76, 78, 82, 71, 71, 71, 44, 44, 54, 58, 78, 78, 78, 66, 61, 61, 58, 54, 47, 54, 69, 54, 71, 80, 58, 58, 35, 10, 38, 53, 53, 44]




---



In [791]:
columns = ['album', 'artists', 'disc_number', 'duration_ms', 'explicit', 'name', 'popularity', 'release_date', 'track_number', 'uri', 'danceability', 'energy', 'key', 'loudness',
'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'followers', 'genres', 'artist_popularity']

In [792]:
columns.sort()
columns

['acousticness',
 'album',
 'artist_popularity',
 'artists',
 'danceability',
 'disc_number',
 'duration_ms',
 'energy',
 'explicit',
 'followers',
 'genres',
 'instrumentalness',
 'key',
 'liveness',
 'loudness',
 'mode',
 'name',
 'popularity',
 'release_date',
 'speechiness',
 'tempo',
 'track_number',
 'uri',
 'valence']

In [793]:
df = pd.DataFrame()

In [794]:
df['acousticness']=acousticness

In [795]:
df['album']=album

In [796]:
df['artist_popularity']=artist_popularity

In [797]:
df['artists']=artists

In [798]:
df['danceability']=danceability

In [799]:
df['disc_number']=disc_number

In [800]:
df['duration_ms']=duration_ms

In [801]:
df['energy']=energy

In [802]:
df['explicit']=explicit

In [803]:
df['followers']=followers

In [804]:
df['genres']=genres

In [805]:
df['instrumentalness']=instrumentalness

In [806]:
df['key']=key

In [807]:
df['liveness']=liveness

In [808]:
df['loudness']=loudness

In [809]:
df['mode']=mode

In [810]:
df['name']=name

In [811]:
df['popularity']=popularity

In [812]:
df['release_date']=release_date

In [813]:
df['speechiness']=speechiness

In [814]:
df['tempo']=tempo

In [815]:
df['track_number']=track_number

In [816]:
df['uri']=uri

In [817]:
df['valence']=valence

In [818]:
df

,acousticness,album,artist_popularity,artists,danceability,disc_number,duration_ms,energy,explicit,followers,genres,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,track_number,uri,valence
0,0.01460,The Infamous,71,Mobb Deep,0.763,1,325506,0.786,True,1179038,east coast hip hop,0.011400,10,0.0817,-6.472,0,"Shook Ones, Pt. II",74,1995-04-25,0.2290,93.857,15,spotify:track:33ZXjLCpiINn8eQIDYEPTD,0.504
1,0.01570,Moment Of Truth,66,Gang Starr,0.532,1,247333,0.819,True,619470,alternative hip hop,0.000000,6,0.3710,-5.282,0,Moment Of Truth,56,1998-03-31,0.2560,90.215,8,spotify:track:1fotoYONO343JjbC8XvPSl,0.594
2,0.20400,A Long Hot Summer,54,Masta Ace,0.729,1,207693,0.619,True,136252,alternative hip hop,0.000000,2,0.0948,-6.798,1,Da Grind (Featuring Apocalypse),52,2004-08-03,0.3860,88.020,5,spotify:track:2ouC5Zp9p98n6kAsXarudl,0.649
3,0.33000,The Predator,76,Ice Cube,0.798,1,260000,0.744,True,3963746,conscious hip hop,0.000106,7,0.2920,-5.328,0,It Was A Good Day,76,1992-11-17,0.1360,82.356,7,spotify:track:2qOm7ukLyHUXWyR4ZWLwxA,0.794
4,0.00776,It Was Written,78,Nas,0.614,1,234093,0.543,True,2115114,conscious hip hop,0.000033,5,0.3830,-7.178,1,The Message,65,1996-07-02,0.3740,178.422,2,spotify:track:1AHfovSnGPVYKaahRtA0U6,0.485
5,0.04460,2001,82,Dr. Dre,0.711,1,304026,0.608,False,5233102,g funk,0.000076,8,0.1230,-4.008,0,The Message,60,1999-11-16,0.0754,82.448,22,spotify:track:6WgxXdBZrtdQFGBcxQxpiv,0.791
6,0.27500,Hell On Earth,71,Mobb Deep,0.875,1,274093,0.658,True,1179038,east coast hip hop,0.000000,0,0.1220,-3.641,1,Hell on Earth (Front Lines),66,1996-11-19,0.3590,92.039,11,spotify:track:1RjU7UhRICmdFPKxCucLgq,0.661
7,0.36600,Murda Muzik,71,Mobb Deep,0.790,1,265493,0.728,True,1179038,east coast hip hop,0.000000,2,0.1120,-9.221,1,Quiet Storm,62,1999-08-17,0.3910,96.978,10,spotify:track:6nwAubYnLGl7F7cDuOoVLp,0.403
8,0.12500,The Infamous,71,Mobb Deep,0.813,1,224533,0.703,True,1179038,east coast hip hop,0.000001,11,0.2370,-5.077,0,Survival of the Fittest,69,1995-04-25,0.2680,94.828,3,spotify:track:7N1Vjtzr1lmmCW9iasQ8YO,0.241
9,0.35700,Rip The Jacker,44,Canibus,0.676,1,293400,0.869,True,62089,battle rap,0.000000,4,0.0467,-2.221,0,No Return,31,2003,0.2310,86.977,5,spotify:track:154o1htY1O9OJBt1gJfrfY,0.839


The following cells check whether your code is correct. Please **don't write any code here**. Just leave them as they are.

In [819]:
# LEAVE BLANK

In [820]:
# LEAVE BLANK

In [821]:
# LEAVE BLANK

In [822]:
# LEAVE BLANK

In [823]:
# LEAVE BLANK

## Saving the data

The final step is to save the DataFrame to a csv file in order to be able to load it into your BigML account. You can do so using DataFrame's ```to_csv``` method. This method takes your DataFrame and saves it to a file of your choosing in the same directoy where your notebook is located.

Run the following cell to save your DataFrame to a .csv file called 'spotify.csv'.

In [824]:
df.to_csv('spotify.csv', sep=',')

<div class="alert alert-danger"><b>Disclaimer: </b>This is a graded assignment. You can share your doubts in the course Forum and give a hand to your classmates with theirs. Yet, remember that posting explicit solutions to any of the exercises is strictly forbidden.</div>